In [1]:
import torch
import torchvision

In [2]:
import arrayfire as af
import array

In [3]:
def toArrayFire(x):
    x_np = x.detach().contiguous().numpy()
    shape = 1
    if len(x_np.shape) == 0:
        shape = (1,)
    else:
        shape = x_np.shape[::-1]
    afArray = af.Array(x_np.ctypes.data, shape, x_np.dtype.char)
    return afArray

def saveStateDict(module, filepath):
    i = 0
    for (name, param) in module.named_parameters():
        #param = module.state_dict()[name]
        print(name, "\t", param.size())
        if 'in_proj' in name:
            print(param.shape)
            q, k, v = param.chunk(3, dim=0)
            print('in_proj!')
            af.array.save_array(name + 'q', toArrayFire(q), filepath, True)
            af.array.save_array(name + 'k', toArrayFire(k), filepath, True)
            af.array.save_array(name + 'v', toArrayFire(k), filepath, True)
            continue
        if len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
    print(i)
    for name in module.state_dict():
        if 'running' in name:
            print(name)
            af_array = toArrayFire(module.state_dict()[name])
            af.array.save_array(name, af_array, filepath + 'running', True)

In [4]:
from models.backbone import *
from models.position_encoding import *
from models.matcher import *

In [5]:
from models.transformer import *

In [20]:
batch_size = 1
embedding_size = 128
src_len = 5
tgt_len = 10
queries = torch.rand(tgt_len, batch_size, embedding_size)
memory = torch.rand(src_len, batch_size, embedding_size)
model = TransformerDecoderLayer(embedding_size, 1, dropout=0.0, dim_feedforward=128)
output = model.forward(queries, memory)

tensor([[[ 1.4640,  0.5245, -0.4588,  ..., -0.0227, -0.3933,  0.8547]],

        [[ 0.8655,  0.2807,  1.1985,  ...,  0.9851,  0.2143, -0.5303]],

        [[ 0.6088,  0.8931, -0.2293,  ..., -0.9108, -0.1970,  1.4215]],

        ...,

        [[-0.0110,  1.6418,  0.6668,  ...,  0.7015, -0.1773, -0.9895]],

        [[-1.3038,  1.0231,  0.1501,  ..., -0.7441,  1.0080,  0.3091]],

        [[-1.1998, -0.3417, -0.0349,  ..., -0.6445,  0.3683, -0.7311]]],
       grad_fn=<NativeLayerNormBackward>)


In [21]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/transformer_decoder_layer.array'
af.array.save_array('queries', toArrayFire(queries), filepath, False)
af.array.save_array('memory', toArrayFire(memory), filepath, True)
i = 2
params = {}
for (name, param) in model.named_parameters():
        if 'in_proj' in name:
            q, k, v = param.chunk(3, dim=0)
            hack = '0'
            if 'in_proj_bias' in name: hack = '1'
            params['0q_' + hack + name] = q
            params['1k_' + hack + name] = k
            params['2v_' + hack + name] = v
            if 'in_proj_bias' in name:
                for key in sorted(params.keys()):
                    af_array = toArrayFire(params[key])
                    if 'weight' in key:
                        af_array = af.array.transpose(af_array)
                    print(key, i, params[key].shape)
                    print(af.array.save_array(key, af_array, filepath, True))
                    i = i + 1
                params = {}
            continue
        elif len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'proj' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'linear' in name:
                af_array = af.array.transpose(af_array)
            print(name, i, param.shape)
            print(af.array.save_array(name, af_array, filepath, True))
            i = i + 1
#af.array.save_array('output', toArrayFire(output), filepath, True)
af.array.save_array('output', toArrayFire(output), filepath, True)

0q_0self_attn.in_proj_weight 2 torch.Size([128, 128])
2
0q_1self_attn.in_proj_bias 3 torch.Size([128])
3
1k_0self_attn.in_proj_weight 4 torch.Size([128, 128])
4
1k_1self_attn.in_proj_bias 5 torch.Size([128])
5
2v_0self_attn.in_proj_weight 6 torch.Size([128, 128])
6
2v_1self_attn.in_proj_bias 7 torch.Size([128])
7
self_attn.out_proj.weight 8 torch.Size([128, 128])
8
self_attn.out_proj.bias 9 torch.Size([128])
9
0q_0multihead_attn.in_proj_weight 10 torch.Size([128, 128])
10
0q_1multihead_attn.in_proj_bias 11 torch.Size([128])
11
1k_0multihead_attn.in_proj_weight 12 torch.Size([128, 128])
12
1k_1multihead_attn.in_proj_bias 13 torch.Size([128])
13
2v_0multihead_attn.in_proj_weight 14 torch.Size([128, 128])
14
2v_1multihead_attn.in_proj_bias 15 torch.Size([128])
15
multihead_attn.out_proj.weight 16 torch.Size([128, 128])
16
multihead_attn.out_proj.bias 17 torch.Size([128])
17
linear1.weight 18 torch.Size([128, 128])
18
linear1.bias 19 torch.Size([128])
19
linear2.weight 20 torch.Size([128, 

28

In [22]:
output

tensor([[[ 1.2040e+00,  2.4613e-01, -5.2788e-01,  ...,  8.7406e-01,
          -8.1491e-01,  6.8629e-01]],

        [[ 1.0051e+00,  3.8374e-01,  1.2483e+00,  ...,  1.6616e+00,
          -9.4458e-02, -6.5259e-01]],

        [[ 6.3987e-01,  7.4390e-01, -3.5206e-01,  ..., -2.2591e-01,
          -5.5494e-01,  1.1156e+00]],

        ...,

        [[-8.5121e-02,  1.4491e+00,  3.2883e-01,  ...,  1.2137e+00,
          -2.5187e-01, -1.0812e+00]],

        [[-1.3683e+00,  1.1406e+00,  2.9048e-01,  ..., -1.3256e-01,
           7.7135e-01,  2.9597e-04]],

        [[-1.4317e+00, -5.8421e-01, -4.9685e-01,  ...,  7.8811e-02,
          -1.9064e-01, -8.7452e-01]]], grad_fn=<NativeLayerNormBackward>)